# Минипроект ч.2
# Исследование данных. Динамика метрик. Прогноз временных рядов fbprophet

## Описание данных
#### Датасет E-Commerce Data  
**Источник** https://www.kaggle.com/datasets/carrie1/ecommerce-data  
**Описание:** 'This is a transnational data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.'

Имеются следующие данные о транзакциях в период с 01.12.2010 по 12.09.2011:

* InvoiceNo — номер транзакции  
* StockCode — код товара  
* Description — описание товара  
* Quantity — количество единиц товара, добавленных в заказ  
* InvoiceDate — дата транзакции   
* UnitPrice — цена за единицу товара  
* CustomerID — id клиента  
* Country — страна, где проживает клиент  
  
Данные содержат в себе записи как об успешных транзакциях, так и об отмененных. В данных встречаются строки с Description 'Manual', которые включают данные об удаленных из чека позициях.

## Задачи минипроекта:

1. Визуализировать основные метрики в динамике за год (выручка, средний чек, возвраты товара, новые\повторные покупатели)
2. Спогнозировать выручку с помощью fbprophet.

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import datetime
from operator import attrgetter
import matplotlib.colors as mcolors
pd.options.mode.chained_assignment = None
from sklearn.cluster import KMeans
import plotly.express as px
import plotly.graph_objs as go
import plotly.io as pio
pio.renderers.default='notebook'

### Загрузка и подготовка данных

In [2]:
df = pd.read_csv('data/archive.zip', encoding='windows-1251')

FileNotFoundError: [Errno 2] No such file or directory: 'data/archive.zip'

In [ ]:
df.info()

In [ ]:
df.duplicated().sum()

In [ ]:
df.loc[df.duplicated()]

В данных есть дубликаты, удалим эти строки.

In [ ]:
df = df.drop_duplicates()

In [ ]:
df.isna().sum()

Также в данных есть пропущенные значения. Так как это значения CustomerID, лучше их удалить: одна из наших целей - разбиение покупателей на сегменты и строки без CustomerID для нас бесполезны.

In [ ]:
df = df.dropna()

Приведем неверно распознанные данные к нужному типу - дату в datetime, а CustomerID в object.

In [ ]:
df.dtypes

In [ ]:
df['CustomerID_ob'] = df['CustomerID'].astype('object').apply(lambda x: str(x).replace('.0', ''))

In [ ]:
df.InvoiceDate = pd.to_datetime(df.InvoiceDate)

In [ ]:
df = df.drop('CustomerID', axis=1)

In [ ]:
df.dtypes

In [ ]:
df.sample(5)

Нам известно, что часть транзакций имеют пометку Manual, и содержат данные об удаленных из чека позициях. Удалим их из нашего датасета.

In [ ]:
df = df[df.Description != 'Manual']

In [ ]:
df = df.reset_index(drop=True)

In [ ]:
df.head()

Проверим количественные данные на выбросы. 

In [ ]:
df.describe()

Мы видим, что в столбце Quantity есть очень большие значения, причем максимум и минимум равны. Очень похоже на ошибочные транзакции, впоследствии отмененные. Проверим, может быть, их несколько. 

In [ ]:
df.Quantity.sort_values()

In [ ]:
df.query('Quantity in (-80995, -74215, -9360, -3114, 12540, 4800, 74215, 80995)').sort_values('CustomerID_ob')

Действительно, 4 самых больших значения Quantity по сути являются сразу же отмененными заказами (возможно, как раз из-за ошибки в количестве). На мой взгляд, для дальнейшего анализа не нужно принимать их во внимание.

In [ ]:
df = df.loc[~df['Quantity'].isin([-80995, -74215, 74215, 80995])]

Теперь обратимся к выбросам столбца UnitPrice. Посмотрим на самые дорогие товары. Мы видим, что это в основном доставки(POSTAGE) и комиссии. Как мы видим, у них есть отличие от товаров - в поле StockCode указаны только буквенные обозначения. При анализе метрик, таких как средний чек, они будут нам только мешать. Удалим эти позиции. А также не будем учитывать транзакции с ценой товара равной нулю.

In [ ]:
df.sort_values('UnitPrice', ascending=False).head(20)

In [ ]:
df = df[~df['StockCode'].str.isalpha()]

In [ ]:
df = df[df.UnitPrice != 0]

Описание итогового датасета

In [ ]:
df.describe()

In [ ]:
df.info()

In [ ]:
df['Revenue'] = df['Quantity'] * df['UnitPrice']

Мы добавили новый столбец с суммой заказа. Это наш датафрейм с очищенными данными.   
Для дальнейшего исследования пока исключим отмененнные заказы. 

In [ ]:
df_purchases = df[(df.Quantity >0)]

In [ ]:
df_purchases.head()

Обратимся к столбцу Country. Мы знаем, что в датасет включены продажи по разным странам. Посмотрим на сумму покупок, количество уникальных заказов и покупателей с точки зрения географии.

In [ ]:
df_purchases.groupby('Country', as_index=False)\
            .agg({'Revenue':'sum', 
                   'InvoiceNo':'nunique', 
                    'CustomerID_ob':'nunique'})\
            .rename(columns={'Revenue':'total_revenue', 
                   'InvoiceNo':'unique_orders', 
                    'CustomerID_ob':'unique_customers'})\
            .sort_values('unique_customers', ascending=False).head(10)

Большая часть данных в датасете относятся к United Kingdom. Вероятно, это внутренний рынок, а на экспорт идет значительно меньше продукции. Посмотрим на соотношение выручки, заказов и покупателей на внутреннем и внешнем рынках. 

In [ ]:
df_purchases['market_group'] = df['Country'].apply(lambda x: 
                                                   'internal_market' if x == 'United Kingdom' 
                                                   else 'foreign_market')

In [ ]:
df_purchases.sample(2)

In [ ]:
df_purchases.groupby('market_group', as_index=False)\
            .agg({'Revenue':'sum', 
                   'InvoiceNo':'nunique', 
                    'CustomerID_ob':'nunique'})\
            .rename(columns={'Revenue':'total_revenue', 
                   'InvoiceNo':'unique_orders', 
                    'CustomerID_ob':'unique_customers'})\
            .sort_values('unique_customers', ascending=False)

Для наших задач - изучения Retention и сегментации пользователей - очевидно интереснее будет проанализировать данные внутреннего рынка. 
Отберем их в финальный датафрейм.

In [ ]:
fin_df = df_purchases[df_purchases.Country == 'United Kingdom'][['InvoiceDate',
                                    'CustomerID_ob', 
                                    'InvoiceNo', 
                                    'StockCode', 
                                    'Description', 
                                    'Quantity',
                                   'UnitPrice',
                                   'Revenue']].reset_index(drop=True)

In [ ]:
fin_df.describe(include='all', datetime_is_numeric=True)

## Динамика метрик в течение года

Основные метрики - Средний чек (AOV), общий объём оборота товаров GMV (Gross Merchandise Value), процент возвратов, repeat rate(доля повторных покупателей), соотношение новых и старых пользователей(Old vs New).

Сформируем таблицу для анализа

In [ ]:
fin_df['orders_date'] = pd.DatetimeIndex(fin_df.InvoiceDate).date

In [ ]:
df_per_day = fin_df.groupby('orders_date').agg(customers_count=('CustomerID_ob', 'nunique'),
                                 orders_count=('InvoiceNo', 'nunique'),
                                 total_revenue=('Revenue', 'sum'))

In [ ]:
df_per_day['avg_check'] = df_per_day.total_revenue / df_per_day.orders_count

In [ ]:
df_per_day['ARPU'] = df_per_day.total_revenue / df_per_day.customers_count

In [ ]:
df_per_day['total_revenue_roll'] = df_per_day.total_revenue.rolling(window=5, min_periods=1).mean()

In [ ]:
fin_df['first_order_date'] = pd.DatetimeIndex(fin_df.groupby('CustomerID_ob').
                                                InvoiceDate.transform('min')).date

In [ ]:
df_per_day['new_customers'] = fin_df.groupby('first_order_date').CustomerID_ob.nunique().sort_index()

In [ ]:
df_per_day = df_per_day.fillna(0)

In [ ]:
df_per_day['old_customers'] = df_per_day.customers_count - df_per_day.new_customers

In [ ]:
df_per_day['repeat_rate'] = df_per_day.old_customers / df_per_day.customers_count * 100

Вспомним про отмененные заказы, создадим отдельный датафрейм и сджойним его с основным.

In [ ]:
cancells_df = df[(df.Quantity <0)& (df.Country == 'United Kingdom')][['InvoiceDate',
                                    'CustomerID_ob', 
                                    'InvoiceNo', 
                                    'StockCode', 
                                    'Description', 
                                    'Quantity',
                                   'UnitPrice',
                                   'Revenue']].reset_index(drop=True)
cancells_df.head(2)

In [ ]:
cancells_df['cancell_date'] = pd.DatetimeIndex(cancells_df.InvoiceDate).date

In [ ]:
cancells_df_per_day = cancells_df.groupby('cancell_date').agg(
                                 customers_cancellation_count=('CustomerID_ob', 'nunique'),
                                 orders_cancellation_count=('InvoiceNo', 'nunique'),
                                 total_cancellation_sum=('Revenue', 'sum'))

In [ ]:
df_per_day = df_per_day.merge(cancells_df_per_day, how='left', left_index=True, right_index=True)

In [ ]:
df_per_day['cancellation_rate'] = df_per_day.customers_cancellation_count / df_per_day.customers_count *100

In [ ]:
df_per_day = df_per_day.reset_index()

In [ ]:
df_per_day['orders_date'] = pd.DatetimeIndex(df_per_day['orders_date'])

In [ ]:
df_per_day = df_per_day.set_index(df_per_day.orders_date)

In [ ]:
df_per_day['avg_check_canc'] = df_per_day.total_cancellation_sum / df_per_day.orders_cancellation_count

Итоговый датафрейм для анализа динамики показателей. Создадим несколько интерактивных визуализаций по основным метрикам, используя Plotly 

In [ ]:
df_per_day.head(3)

In [ ]:
fig = px.bar(df_per_day.resample('w').mean(),  
             y=['total_revenue', 'total_cancellation_sum'],
            title='Gross Merchandise Value VS Loss from order cancellation per weeks')
fig.update_layout(
    margin=dict(
    autoexpand=True
    ),
    legend=dict(
        yanchor="top",
        y=1.03,
        xanchor="left",
        x=0.01
    ),
    legend_title_text='',
    yaxis=dict(
        title='', showticklabels=False
    ),
    xaxis=dict(
        dtick="M1",
    tickformat="%b\n%Y",
        title='Date of order'
    ),
    showlegend=True,
    plot_bgcolor='white', 
    uniformtext_minsize=8,
    uniformtext_mode='hide')
newnames = {'total_revenue': "Gross Merchandise Value, $", 
            'total_cancellation_sum': "Loss from order cancellation, $"}
fig.for_each_trace(lambda t: t.update(
    name = newnames[t.name],
    legendgroup = newnames[t.name],
    hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])))
fig.show('notebook')
fig.write_html('notebook.html')

In [ ]:
fig = px.bar(df_per_day.resample('w').mean(),  y=['avg_check', 'avg_check_canc'], 
            title='Average Order Value VS Average Cancelled Order Value per weeks')
fig.update_layout(margin=dict(
        autoexpand=True
    ),
    legend=dict(
        yanchor="top",
        y=1.03,
        xanchor="left",
        x=0.01
    ),
    legend_title_text='',
    yaxis=dict(
        title='', showticklabels=False
    ),
    xaxis=dict(
        dtick="M1",
    tickformat="%b\n%Y",
        title='Date of order'
    ),
    showlegend=True,
    plot_bgcolor='white', 
    uniformtext_minsize=8,
                  uniformtext_mode='hide')
newnames = {'avg_check': "Average Order Value, $", 'avg_check_canc': "Average Cancelled Order Value, $"}
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
fig.show('notebook')
fig.write_html('notebook.html')

In [ ]:

fig = px.bar(df_per_day.resample('m').sum(),  
             y=['new_customers','old_customers'], 
            text_auto=True, 
             title="New customers VS old customers")
fig.update_layout(margin=dict(
        autoexpand=True),
    showlegend=True, 
    legend=dict(
        yanchor="top",
        y=1.03,
        xanchor="left",
        x=0.01
    ),
    legend_title_text='',
    yaxis=dict(
        title='Number of customers',
        titlefont_size=16,
        tickfont_size=14, visible=True, showticklabels=False
    ), 
    xaxis=dict(
        dtick="M1",
    tickformat="%b\n%Y",
        title='Order\'s month'
    ),
                  
    plot_bgcolor='white', 
    uniformtext_minsize=8,
                  uniformtext_mode='hide',
    title_font_color="black")
fig.update_traces(textposition='outside')
newnames = {'new_customers': "New customers", 'old_customers': "Old customers"}
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
fig.show('notebook')
fig.write_html('notebook.html')

In [ ]:
fig = px.line(df_per_day.resample('w').mean(),  y=['repeat_rate', 
                                                   'cancellation_rate'], 
              markers=True, line_shape='spline', title='Percent of reodering and refusing customers')
fig.update_layout(margin=dict(
        autoexpand=True),
    legend=dict(
        yanchor="top",
        y=1.03,
        xanchor="left",
        x=0.01
    ),
    legend_title_text='',
    yaxis=dict(
        title='Percentage',
        titlefont_size=14,
        tickfont_size=14, visible=True, showticklabels=True
    ),
    xaxis=dict(
        dtick="M1",
    tickformat="%b\n%Y",
        title='Date of order'
    ),
    showlegend=True,
    plot_bgcolor='white', 
    uniformtext_minsize=8,
    uniformtext_mode='hide')
newnames = {'repeat_rate': "Percent of repeat customers", 
            'cancellation_rate': "Percent of customers with cancelled orders"}
fig.for_each_trace(lambda t: t.update(name = newnames[t.name],
                                      legendgroup = newnames[t.name],
                                      hovertemplate = t.hovertemplate.replace(t.name, newnames[t.name])
                                     )
                  )
fig.show('notebook')
fig.write_html('notebook.html')

В целом, дела у магазина идут хорошо - оборот увеличивается, причем в основном за счет увеличения процента повторных покупателей. Рост особенно выражен в последние 3 месяца, начиная с сентября. (Помним, что данные только за часть последнего месяца года, так что эти данные не принимаем во внимание).  

На что, возможно, стоило бы обратить внимание - на увеличение среднего чека (он тоже растет, но не так уверенно) и привлечение новых клиентов (в течение года их число скорее снижается).


## Прогноз Gross Merchandise Value с fbprophet

In [ ]:
from datetime import date
import holidays
from fbprophet import Prophet

Подготовим датафрейм для прогноза

In [ ]:
df_pr = df_per_day[['orders_date', 'total_revenue_roll']]\
            .loc[df_per_day['orders_date'] > '2010-01-01']\
            .reset_index(drop=True)\
            .rename({'orders_date':'ds', 'total_revenue_roll':'y'}, axis ='columns')

Импортируем праздники 

In [ ]:
holidays_dict = holidays.UK(years=(2010, 2011, 2012))

In [ ]:
df_holidays = pd.DataFrame.from_dict(holidays_dict, orient='index') \
    .reset_index().rename({'index':'ds', 0:'holiday'}, axis ='columns')

In [ ]:
df_holidays['ds'] = pd.to_datetime(df_holidays.ds)


In [ ]:
df_holidays = df_holidays.sort_values(by=['ds']).reset_index(drop=True)
df_holidays.tail()

Определим тестовую выборку из датафрейма - последние 30 дней

In [ ]:
predictions = 30
train_df = df_pr[:-predictions]

In [ ]:
train_df.head()

Настроим prophet - с учетом праздников, недельной и годовой сезонности.

In [ ]:
m = Prophet(holidays=df_holidays, 
            daily_seasonality=False, 
            weekly_seasonality=True, 
            yearly_seasonality=True,
           changepoint_prior_scale=0.02)
m.add_country_holidays(country_name='UK')
m.fit(train_df)

Предсказываем 30 дней

In [ ]:
future = m.make_future_dataframe(periods=predictions)
forecast = m.predict(future)

In [ ]:
forecast.tail(1)

Посмотрим на результат на графике

In [ ]:
from prophet.plot import plot_plotly, plot_components_plotly
plot_plotly(m, forecast)

Также мы можем посмотреть на сезонность данных

In [ ]:
plot_components_plotly(m, forecast)

In [ ]:
# Рисуем график с границами прогноза
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
init_notebook_mode(connected = True)

iplot([
    go.Scatter(x=df_pr['ds'], y=df_pr['y'], name='fact'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat'], name='prediction'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat_upper'], fill='tonexty', mode='none', name='upper'),
    go.Scatter(x=forecast['ds'], y=forecast['yhat_lower'], fill='tonexty', mode='none', name='lower'),
    go.Scatter(x=forecast['ds'], y=forecast['trend'], name='trend')
])
fig.write_html('notebook.html')

Проверим качество прогноза

In [ ]:
cmp_df = forecast.set_index('ds')[['yhat', 'yhat_lower', 'yhat_upper']].join(df_pr.set_index('ds'))
cmp_df['e'] = cmp_df['y'] - cmp_df['yhat']
cmp_df['p'] = 100*cmp_df['e']/cmp_df['y']
print('MAPE (средняя абсолютная ошибка в процентах) – ', np.mean(abs(cmp_df[-predictions:]['p'])),'%')
print('MAE (средняя абсолютная ошибка) – ', np.mean(abs(cmp_df[-predictions:]['e'])))

Рассчитаем прогноз на полный период (например, год) с теми же параметрами.

In [ ]:
prediction_days = 365
final_train_df = df_pr
f = Prophet(holidays=df_holidays, 
            daily_seasonality=False, 
            weekly_seasonality=True, 
            yearly_seasonality=False,
           n_changepoints=20)
f.add_country_holidays(country_name='UK')
f.fit(final_train_df)
final_future = f.make_future_dataframe(periods=prediction_days)
final_forecast = f.predict(final_future)

In [ ]:
f.plot(final_forecast);
sns.despine()

In [ ]:
# Рисуем график с границами прогноза на полном периоде
iplot([
    go.Scatter(x=df_pr['ds'], y=df_pr['y'], name='fact'),
    go.Scatter(x=final_forecast['ds'], y=final_forecast['yhat'], name='yhat'),
    go.Scatter(x=final_forecast['ds'], y=final_forecast['yhat_upper'], fill='tonexty', mode='none', name='upper'),
    go.Scatter(x=final_forecast['ds'], y=final_forecast['yhat_lower'], fill='tonexty', mode='none', name='lower'),
    go.Scatter(x=final_forecast['ds'], y=final_forecast['trend'], name='trend')
])
fig.write_html('notebook.html')

Также prophet позволяет нам определить точки изменения - это точки даты и времени, в которых временные ряды имеют резкие изменения траектории. Построим вертикальные линии, где произошли потенциальные точки изменения

In [ ]:
from fbprophet.plot import add_changepoints_to_plot
fig = f.plot(forecast)
a = add_changepoints_to_plot(fig.gca(), f, final_forecast)

In [ ]:
f.changepoints

Выгружаем прогноз в эксель. Спрогнозированное значение лежит в столбце yhat

In [ ]:
final_forecast.to_excel("./app_forecast.xlsx", sheet_name='Data', index=False, encoding="cp1251")